In [ ]:
# Intialize the tensorflow-gpu <-> physical matching

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from load.load_data import load_opendata
from load.load_data import load_mydata

import matplotlib.pyplot as plt
%matplotlib inline
import random

In [ ]:
openloader = load_opendata()
X_open, Y_open = openloader.load_data(classifier_label=None, dsize=(64,64), comp_ratio=4)

myloader = load_mydata()
X_my, Y_my = myloader.load_data(classifier_label="OOP", dsize=(64,64), comp_ratio=10, verbose=0)
X_my_test, Y_my_test = myloader.load_test_data(classifier_label="OOP", dsize=(64,64), comp_ratio=4, verbose=0)

print("Open data is ", len(X_open),", My Train set is ", len(Y_my),  ",  My Test set is ", len(Y_my_test))

In [ ]:
import numpy as np
from utils.process import preprocess, noise, augment, tsne_plot, display_pair, shuffler

# Normalized Image
X = preprocess(X_open, shape = (64,64,3))

# Data Augmentation (5%*3=15%)
X = augment(X, ratio=0.05)

# Shuffled Image
X = shuffler(X)

# Noise Image
Noise_X = noise(X)

In [ ]:
display_pair(X,Noise_X)

In [ ]:
# Train, Val, Test Split
X_train = X[:int(len(X)*0.9)]
X_val = X[int(len(X)*0.9):]

print(X_train.shape)
print(X_val.shape)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
import model.model_autoencoder
from model.model_autoencoder import model_autoencoder
import importlib
importlib.reload(model.model_autoencoder)

import tensorflow as tf
import os


def scheduler(epoch, lr):
    if epoch in [5,9,12,20]:
        lr = 0.1*lr
    return lr
    
autoencoder = model_autoencoder(input_shape = (64, 64, 3))
adam = Adam(learning_rate=0.001)
ls_callback = LearningRateScheduler(scheduler)
es_callback = EarlyStopping(monitor='val_loss', patience=3)
filepath = os.path.join(os.getcwd(), "ckpt/", "Autoencoder_fine-{epoch:01d}-{val_loss:.2f}.h5")
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

autoencoder.compile(loss = 'mse', optimizer = adam)
autoencoder.summary()

In [ ]:
autoencoder.fit(x=X_train, y = X_train, epochs=30, batch_size=32, shuffle=True, validation_data=(X_val, X_val), callbacks=[ls_callback, es_callback, checkpoint])

In [ ]:
# Save Model
autoencoder.save('./ckpt/model_oop_autoencoder_fine')

In [ ]:
# Display the train data and a version of it with added noise
predictions = autoencoder.predict(Noise_X[len(Noise_X)*0.9:])

In [ ]:
display_pair(Noise_X[len(Noise_X)*0.9:], predictions)